# 3. Harmonize Raster Data and Create Model

In [6]:
# Download stored variables from previous notebooks

# Variables from study area download
# Project data directory
%store -r habitat_suitability_data_dir 
# Site gdfs
%store -r comanche_grassland_gdf pawnee_grassland_gdf 

# Variables from Polaris download


# Variables from SRTM download
%store -r srtm_comanche_result_da srtm_pawnee_result_da srtm_da_results slope_da slope_da_list

# Variables from the MACA v2 download 
%store -r maca_df maca_da_list

# Harmonize Data Layers: Ensure that each raster layer has 
# consistent spatial resolution and projection.


“””To harmonize data layers, ensure that all raster layers have the 
same spatial resolution and projection. The function `rio.reproject_match()` 
from the `rioxarray` library can be used to reproject and align one raster 
layer to match the resolution, extent, and CRS of a reference raster layer. 
Use `ds.rio.reproject_match()` to align all raster grids.””


In [7]:
# Prepare for download Part 1 of 1
## Import packages that will help with...

# Reproducible file paths
import os # Reproducible file paths
from glob import glob  # returns list of paths
import pathlib # Find the home folder

# Find files by pattern
import matplotlib.pyplot as plt # Overlay pandas (vector) and xarry (raster) plots 
import rioxarray as rxr # Work with geospatial raster data

# Work with tabular, vector, and raster data
import geopandas as gpd # work with vector data
import numpy as np # numerical computing
import pandas as pd # Group and aggregate
from rioxarray.merge import merge_arrays # Merge rasters
import xarray as xr # Adjust images

In [8]:
# Prep for harmonizing Part 1 of 2
# Create output directory for this harmonized raster
harmonized_raster_dir = os.path.join(habitat_suitability_data_dir, 'harmonized_rasters')
os.makedirs(harmonized_raster_dir, exist_ok=True) 

In [ ]:
# Prep for harmonizing Part 2 of 2
# Reminder for site variables

# POLARIS 
# 

# SRTM 

# MACA


In [ ]:
# Create function to harmonize the 3 raster layers
def harmonize_raster_layers(reference_rasters, reference_raster, input_rasters, output_dir):
    """
    Harmonize raster layers to ensure consistent spatial resolution and projection.

    Args:
        reference_raster (str): Path to the reference raster file that other rasters will match.
        input_rasters (list): List of paths to the input raster files to harmonize.
        output_dir (str): Directory to save the harmonized raster files.

    Returns:
        list: List of file paths to the harmonized raster files.
    """

    # Iterate through multiple sites or study areas
    for site_name, site_gdf in {
    'comanche': reference_raster,'pawnee':reference_raster}.items():


            # Load the reference raster 
            ref_raster = rxr.open_rasterio(reference_raster, masked=True)

            # Create list to save data arrays back to
            harmonized_files = []

            # Create for loop to iterate through multiple rasters
            for raster_path in input_rasters:
                # Load the input raster
                input_raster = rxr.open_rasterio(raster_path, masked=True)

                # Reproject and align the input raster to match the reference raster
                harmonized_raster = input_raster.rio.reproject_match(ref_raster)

    # Save the harmonized raster to the output directory
    output_file = os.path.join(output_dir, os.path.basename(raster_path))
    harmonized_raster.rio.to_raster(output_file)

    print(f"Harmonized raster saved to: {output_file}")
    harmonized_files.append(output_file)

    return harmonized_files

In [ ]:
# Test the harmonized raster_layers function
harmonize_raster_layers(maca_df, , harmonized_raster_dir)